In [171]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [172]:
file = open('text.txt', 'r')
text = file.read()
file.close()
tokens = [int(token) for token in text.encode()]
print(f"Length of text: {len(text)}")
print(f"Length of tokens: {len(tokens)}")
# print(f"Text: {text}")
# print(f"Tokens: {tokens}")

Length of text: 23269
Length of tokens: 24531


In [173]:
def getbpc(bytes):
    bpc = dict()
    for b1, b2 in zip(bytes, bytes[1:]):
        cnt = bpc.get((b1, b2), 0)
        bpc[(b1, b2)] = cnt+1
    return bpc

def replace(tokens, pair, newid):
    toknew = []
    i = 0
    while i < len(tokens):
        if i == len(tokens) - 1: toknew.append(tokens[i]); break
        if tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            toknew.append(newid); i += 2
        else: toknew.append(tokens[i]); i += 1
    return toknew

In [184]:
def shrink(vs, tokens, show_stats = False):
    if show_stats:
        tokcntstat = np.zeros(vs - 255)
        lenstat = np.zeros(vs - 255)
        diffstat = np.zeros(vs - 255)
    tokcnt = 255
    merges = {}
    tokentemp = tokens.copy()
    while tokcnt < vs:
        bpc = getbpc(tokentemp)
        pair = max(bpc, key=bpc.get)
        tokcnt += 1
        merges[pair] = tokcnt
        toknew = replace(tokentemp, pair, tokcnt)
        if show_stats:
            tokcntstat[vs - tokcnt] = tokcnt
            lenstat[vs - tokcnt] = len(toknew)
            diffstat[vs - tokcnt] = len(tokentemp) - len(toknew)
        tokentemp = toknew
    if show_stats:
        plt.figure()
        plt.plot(tokcntstat, diffstat)
        compressionRatios = len(tokens) / lenstat
        plt.figure()
        plt.plot(tokcntstat, lenstat)
        plt.figure()
        plt.plot(tokcntstat, (compressionRatios))
    return (tokentemp, merges)

In [187]:
stokens, merges = shrink(300, tokens, show_stats=False)

In [194]:
vocab = {i: bytes([i]) for i in range(256)}
for (a, b), c in merges.items():
    vocab[c] = vocab[a] + vocab[b]
    # print(vocab[c])

In [199]:
def decode(ids):
    decoded = []
    for i in ids:
        decoded += vocab[i]
    return decoded

str(bytes(decode(stokens)), encoding="utf-8")

'Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.\n\nA few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.\n\nI’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those 